In [1]:
import lumipy as lm
from lusidjam import RefreshingToken as rt

atlas = lm.get_atlas(token=rt())

In [2]:
portfolios = atlas.lusid_portfolio()

pf_codes = portfolios.select(portfolios.portfolio_code).where(
    (portfolios.portfolio_scope == 'Finbourne-Examples') &
    (portfolios.portfolio_code.like('%equ%')) &
    (portfolios.portfolio_code.not_like('%swap%'))
).go().PortfolioCode.tolist()

pf_codes

['UK-Equities', 'Global-Equity']

# Tutorial 6 - Unions

In this tutorial we'll build up a more complicated query where we use `union` concatenate a collection of table variable subqueries each of which depend on a scalar variable. 

In [3]:
holding = atlas.lusid_portfolio_holding()

In [4]:
def subquery(pf_code):

    total_cost = holding.select(
        TotalCost=holding.cost_amount_portfolio_currency.sum()
    ).where(
        (holding.portfolio_code == pf_code) & 
        (holding.portfolio_scope == 'Finbourne-Examples')
    ).to_scalar_var()

    cost_table = holding.select(
        holding.portfolio_code,
        holding.lusid_instrument_id, 
        holding.cost_amount_portfolio_currency,
        CostFractionPc=100*holding.cost_amount_portfolio_currency/total_cost
    ).where(
        (holding.portfolio_code == pf_code) & 
        (holding.portfolio_scope == 'Finbourne-Examples')
    ).order_by(
        (holding.cost_amount_portfolio_currency/total_cost).descending()
    ).limit(
        5
    ).to_table_var()
    
    return cost_table.select('*')


In [5]:
qry = lm.concat(map(subquery, pf_codes))
df = qry.go()
df

,PortfolioCode,LusidInstrumentId,CostAmountPortfolioCurrency,CostFractionPc
0,Global-Equity,LUID_1W5WYN3O,1.760183e+09,20.114573
1,Global-Equity,LUID_4CINGMZM,1.322168e+09,15.109133
2,Global-Equity,LUID_GJ9TICS4,1.050759e+09,12.007598
3,Global-Equity,LUID_J4G8D0DV,1.146539e+09,13.102119
4,Global-Equity,LUID_JTQY6QFI,1.822715e+09,20.829161
5,UK-Equities,LUID_941RZG0H,2.871400e+05,2.965447
6,UK-Equities,LUID_ARH76A7B,2.314375e+05,2.390177
7,UK-Equities,LUID_E9EW4TBO,3.868000e+05,3.994688
8,UK-Equities,LUID_KJFBD4GQ,5.552418e+06,57.342760
9,UK-Equities,LUID_LH6UUF74,2.585520e+06,26.702035


In [6]:
# Print the underlying SQL
qry.print_sql()

@@sv_429244219210369233 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] = 'UK-Equities') and ([PortfolioScope] = 'Finbourne-Examples');

@@sv_63693781723779070 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] = 'Global-Equity') and ([PortfolioScope] = 'Finbourne-Examples');

@tv_63921535230829207 = 
select
  [PortfolioCode], [LusidInstrumentId], [CostAmountPortfolioCurrency], ([CostAmountPortfolioCurrency] * 100) / @@sv_429244219210369233 as [CostFractionPc] 
from
  Lusid.Portfolio.Holding 
where
  ([PortfolioCode] = 'UK-Equities') and ([PortfolioScope] = 'Finbourne-Examples') 
order by
  [CostAmountPortfolioCurrency] / @@sv_429244219210369233 desc 
limit
  5;

@tv_74848762479102352 = 
select
  [PortfolioCode], [LusidInstrumentId], [CostAmountPortfolioCurrency], ([CostAmountPortfolioCurrency] * 100) / @@sv_63693781723779070 as [CostFractionP

## Union Types

There are two types of union available: `union` (like above) and `union all` both will concatenate tables of data together along the column axis but the former will filter duplicate rows.